# PoseSliders

Running this cell should allow you to control the 6 DOF pose of a mustard bottle in drake visualizer.

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import numpy as np

from pydrake.common import FindResourceOrThrow
from pydrake.common.value import AbstractValue
from pydrake.geometry import ConnectDrakeVisualizer, FramePoseVector, SceneGraph
from pydrake.math import RigidTransform
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.multibody.tree import BodyIndex
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem

from pydrake.systems.jupyter_widgets import PoseSliders


class PoseToFramePoseVector(LeafSystem):
    """
    Wraps a single pose input into a FramePoseVector.
    """

    def __init__(self, frame_id):
            LeafSystem.__init__(self)
            self.frame_id = frame_id
            self.DeclareAbstractInputPort(
                "pose", AbstractValue.Make(RigidTransform.Identity()))
            self.DeclareAbstractOutputPort(
                "vector", lambda: AbstractValue.Make(FramePoseVector()), self.CalcOutput)

    def CalcOutput(self, context, output):
            pose = self.EvalAbstractInput(context, 0).get_value()
            output.get_value().set_value(id=frame_id, value=pose)


builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
plant = MultibodyPlant(time_step=0.0)
plant.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant, scene_graph).AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf"))
plant.Finalize()

frame_id = plant.GetBodyFrameIdOrThrow(
    plant.GetBodyByName("base_link_mustard").index())

# Add pose sliders.
sliders = builder.AddSystem(PoseSliders(visible=[True] * 6))
to_vector = builder.AddSystem(PoseToFramePoseVector(frame_id))
builder.Connect(sliders.get_output_port(0), to_vector.get_input_port(0))
builder.Connect(
        to_vector.get_output_port(0),
        scene_graph.get_source_pose_port(plant.get_source_id()))

vis = ConnectDrakeVisualizer(builder, scene_graph)

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_context()

# Change this to `np.inf` if you want the sim to run forever.
t_max = 5.

simulator.set_target_realtime_rate(1.0)
stop_button = widgets.Button(description="Stop Simulation")
done = False

def on_stop(_):
    global done
    done = True

stop_button.on_click(on_stop)
display(stop_button)

while not done and context.get_time() < t_max:
    simulator.AdvanceTo(context.get_time() + 0.1)
print("Simulation Done")